# Audio preprocessing 
**pipeline:**


1.   Load a file
2.   pad the signal (if necessary)
3.   DSP: extracting log spectogram from signal
4.   normalize spectogram
5.   save the normalized spectogram

the previous steps where the preprocessing steps for every audio file, so we will create a class for performing these tasks which is audio **preprocessing**




## Creating loader class
for following clean code standards we will separate each function into a class.

the first class will be the loader class

In [1]:
import librosa
import numpy as np
import os
import pickle

In [2]:
#a Loader class is responsible for loading an audio file
#built with help of librosa
class Loader:
  def __init__(self, sample_rate, duration, mono):
    self.sample_rate = sample_rate
    self.duration = duration
    self.mono = mono
  
  def load(self, file_path):
    signal, sample_rate = librosa.load(file_path,sr= self.sample_rate,mono=self.mono, duration=self.duration)
    return (signal,sample_rate)

## Creating Padder Class
built with help of numpy<br> 
Modes:


1.   **constant**: Pads with a constant value default is zero.
2.  **edge**:Pads with the edge values  of array.
3.  **empty**:Pads with undefined value
4.  **mean**:Pads with the mean value of all or part of the vector along each axis.

[more](https://numpy.org/doc/stable/reference/generated/numpy.pad.html)




In [3]:
#a Padding Class is responsible for padding an array
class Padder:
  def __init__(self, mode="constant"):
    self.mode = mode
  
  def left_pad(self, array, num_missing_item):
    padded_array = np.pad(array, (num_missing_item,0),self.mode)
    return padded_array

  def right_pad(self, array, num_missing_item):
    padded_array = np.pad(array, (0,num_missing_item),self.mode)
    return padded_array

## Building LogSpectogramExtractor



In [4]:
#a logspectogramextractor is responsible for extracting log spectogram
class LogSpectorgramExtracor:
  def __init__(self, frame_size, hop_length):
    self.frame_size = frame_size 
    self.hop_length = hop_length 
  
  def extract(self, signal):
    stft = librosa.stft(signal, n_fft = self.frame_size, hop_length=self.hop_length)[:-1]
    # ( 1   +frameSize/2*num of frame) reutrns odd number, hence we will ignore last element
    spectogram = np.abs(stft)
    log_spectogram= librosa.amplitude_to_db(spectogram)
    return log_spectogram


## Building MinMaxNormalizer
Min-max normalization (usually called feature scaling) performs a linear transformation on the original data. This technique gets **all the scaled data in the range (0, 1)**. The formula to achieve this is the following: Min-max normalization preserves the relationships among the original data values.

Xscaled= X-Xmin / Xmax- Xmin

**at Xscaled = Xmax:**

&nbsp;&nbsp;&nbsp;Xscaled = Xmax - Xmin / Xmax - Xmin = 1 (maximum value)

**at Xscaled = Xmin:**

&nbsp;&nbsp;&nbsp;Xscaled = Xmin - Xmin / Xmax - Xmin = 0 (minimum value)


**When should I use min-max normalization? Normalization (Min-Max Scalar)** 

Normalization is useful in models such as **k-nearest neighbors and artificial neural networks**, or anywhere where the data we are using has **varying scales or precision** 

**for example:**
consider a dataset that has two features **(salary and years worked)**. Both of these features will have high differences between their values. For example, salary might be a thousand times bigger than years worked – normalization makes sure the entire data in the same range.

**Note:**

Not every dataset needs normalization in Machine Learning. It is only required when data features have varying ranges.

In [5]:
#a MinMaxNormalizer is responsible for the normalization of an array
class MinMaxNormalizer:
  def __init__(self, min_val, max_val):
    self.min_val = min_val
    self.max_val = max_val
  
  def normlize(self, array):
    norm_array = (array - array.min()) / (array.max() -  array.min())
    norm_array = norm_array* (self.max_val -  self.min_val) +  self.min_val
    #return normalized array between self.max and self.min rather than 0 and 1

    return norm_array
  
  def denormlize(self, norm_array, original_min, original_max):
    array = (norm_array - self.min_val)/(self.max_val - self.min_val)
    array =  array * (original_max - original_min) + original_min
    return array
   

## Building Saver
Python pickle module is used for serializing and de-serializing a Python object structure.** Any object in Python can be pickled** so that it can be **saved on disk.**

The **serialization** process is a way to convert** a data structure** into a **linear form** that can be stored or transmitted over a network.

What pickle does is that it “serializes” the object first before writing it to file. Pickling is a way to convert a python object (list, dict, etc.) into a character stream. The idea is that this character stream contains all the information necessary to reconstruct the object in another python script.


**The Python pickle module basically consists of four methods:**

`pickle.dump(obj, file, protocol=None, *, fix_imports=True, buffer_callback=None)`

`pickle.dumps(obj, protocol=None, *, fix_imports=True, buffer_callback=None)`

`pickle.load(file, *, fix_imports=True, encoding="ASCII", errors="strict", buffers=None)`

`pickle.loads(bytes_object, *, fix_imports=True, encoding="ASCII", errors="strict", buffers=None)`

The first two methods are used during the **pickling process,** and the other two are used during unpickling. The only difference between dump() and dumps() is that the first creates a file containing the serialization result, whereas the second returns a string.

To differentiate dumps() from dump(), it’s helpful to remember that the s at the end of the function name stands for string. The same concept also applies to load() and loads(): The first one reads a file to start the unpickling process, and the second one operates on a string.

[more](https://realpython.com/python-pickle-module/)

In [6]:
# a Saver class is responsible to save features and the min, max values
class Saver:
    def __init__(self,feature_save_dir, min_max_save_dir):
      self.feature_save_dir = feature_save_dir
      self.min_max_save_dir = min_max_save_dir
    
    def save_feature(self, feature, file_path):
      save_path = self._generate_save_path(file_path)
      np.save(save_path, feature)
      return save_path
    
    def _generate_save_path(self, file_path):
      file_name = os.path.split(file_path)[1] #the name
      save_path = os.path.join(self.feature_save_dir,file_name + ".npy" )
      return save_path
    
    def save_min_max_values(self,min_max_values):
      save_path = os.path.join(self.min_max_save_dir, "min_max_values.pkl")
      self._save(min_max_values, save_path)
    
    @staticmethod
    def _save(min_max_values, save_path): #serlize the map and save it
      with open(save_path, "wb") as f :
        pickle.dump(min_max_values, f)




## Implementing preprocessing pipeline
it will combine the previous classes 

In [7]:
class PreprocessingPipeline:
  def __init__(self):
    self.padder =None
    self.extractor = None
    self.normaliser = None
    self.saver = None
    self.min_max_values = {}
    self._loader =None
    self._num_expected_sampled = None
  
  @property
  def loader(self):
    return self._loader

  @loader.setter
  def loader(self, loader):
    self._loader = loader
    self._num_expected_sampled = int(loader.sample_rate* loader.duration)

  def process(self, audio_files_dir):
    for root, subroots, files in os.walk(audio_files_dir):
      for file in files: #file name
        file_path = os.path.join(root, file)
        self._process_file(file_path)
        print(f"Proccessed file {file_path}")
    self.saver.save_min_max_values(self.min_max_values)
    print("saved min max")
  
  def _process_file(self, file_path):
    signal, sr = self._loader.load(file_path) #load signal
    if self._is_padding_necessary(signal):
      signal = self._apply_padding(signal)
    feature = self.extractor.extract(signal)
    norm_feature = self.normaliser.normlize(feature)
    save_path = self.saver.save_feature(norm_feature, file_path)
    self._store_min_max_values(save_path, feature.min(), feature.max())

  def _is_padding_necessary(self, signal):
    if len(signal)< self._num_expected_sampled:
      return True
    return False

  def _apply_padding(self, signal):
    num_of_padded_sample =self._num_expected_sampled - len(signal)
    padded_signal = self.padder.right_pad(signal, num_of_padded_sample)
    return padded_signal
    
  def _store_min_max_values(self, save_path, min_val, max_val):
    print(f"saved at {save_path}")
    self.min_max_values[save_path] = {
        "min" :min_val,
        "max": max_val
    }

## Runing an object from the class

In [8]:
if __name__ == "__main__":
    FRAME_SIZE = 512
    HOP_LENGTH = 256
    DURATION = 0.74  # in seconds
    SAMPLE_RATE = 22050
    MONO = True

    
    SPECTROGRAMS_SAVE_DIR = "./spectrograms/"
    MIN_MAX_VALUES_SAVE_DIR = "./"
    FILES_DIR = "./audio/"
    
    loader = Loader(SAMPLE_RATE,DURATION, MONO )
    padder = Padder()
    log_spectogram_extractor = LogSpectorgramExtracor(FRAME_SIZE, HOP_LENGTH)
    min_max_normlizer = MinMaxNormalizer(0, 1)
    saver = Saver(SPECTROGRAMS_SAVE_DIR, MIN_MAX_VALUES_SAVE_DIR)

    preprocessing_pipeline = PreprocessingPipeline()

    preprocessing_pipeline.loader = loader
    preprocessing_pipeline.padder = padder
    preprocessing_pipeline.extractor = log_spectogram_extractor
    preprocessing_pipeline.normaliser = min_max_normlizer
    preprocessing_pipeline.saver = saver

    preprocessing_pipeline.process(FILES_DIR)


saved at ./spectrograms/0_george_0.wav.npy
Proccessed file ./audio/0_george_0.wav
saved at ./spectrograms/0_george_1.wav.npy
Proccessed file ./audio/0_george_1.wav
saved at ./spectrograms/0_george_10.wav.npy
Proccessed file ./audio/0_george_10.wav
saved at ./spectrograms/0_george_11.wav.npy
Proccessed file ./audio/0_george_11.wav
saved at ./spectrograms/0_george_12.wav.npy
Proccessed file ./audio/0_george_12.wav
saved at ./spectrograms/0_george_13.wav.npy
Proccessed file ./audio/0_george_13.wav
saved at ./spectrograms/0_george_14.wav.npy
Proccessed file ./audio/0_george_14.wav
saved at ./spectrograms/0_george_15.wav.npy
Proccessed file ./audio/0_george_15.wav
saved at ./spectrograms/0_george_16.wav.npy
Proccessed file ./audio/0_george_16.wav
saved at ./spectrograms/0_george_17.wav.npy
Proccessed file ./audio/0_george_17.wav
saved at ./spectrograms/0_george_18.wav.npy
Proccessed file ./audio/0_george_18.wav
saved at ./spectrograms/0_george_19.wav.npy
Proccessed file ./audio/0_george_19.

saved at ./spectrograms/0_lucas_17.wav.npy
Proccessed file ./audio/0_lucas_17.wav
saved at ./spectrograms/0_lucas_18.wav.npy
Proccessed file ./audio/0_lucas_18.wav
saved at ./spectrograms/0_lucas_19.wav.npy
Proccessed file ./audio/0_lucas_19.wav
saved at ./spectrograms/0_lucas_2.wav.npy
Proccessed file ./audio/0_lucas_2.wav
saved at ./spectrograms/0_lucas_20.wav.npy
Proccessed file ./audio/0_lucas_20.wav
saved at ./spectrograms/0_lucas_21.wav.npy
Proccessed file ./audio/0_lucas_21.wav
saved at ./spectrograms/0_lucas_22.wav.npy
Proccessed file ./audio/0_lucas_22.wav
saved at ./spectrograms/0_lucas_23.wav.npy
Proccessed file ./audio/0_lucas_23.wav
saved at ./spectrograms/0_lucas_24.wav.npy
Proccessed file ./audio/0_lucas_24.wav
saved at ./spectrograms/0_lucas_25.wav.npy
Proccessed file ./audio/0_lucas_25.wav
saved at ./spectrograms/0_lucas_26.wav.npy
Proccessed file ./audio/0_lucas_26.wav
saved at ./spectrograms/0_lucas_27.wav.npy
Proccessed file ./audio/0_lucas_27.wav
saved at ./spectro

saved at ./spectrograms/0_theo_16.wav.npy
Proccessed file ./audio/0_theo_16.wav
saved at ./spectrograms/0_theo_17.wav.npy
Proccessed file ./audio/0_theo_17.wav
saved at ./spectrograms/0_theo_18.wav.npy
Proccessed file ./audio/0_theo_18.wav
saved at ./spectrograms/0_theo_19.wav.npy
Proccessed file ./audio/0_theo_19.wav
saved at ./spectrograms/0_theo_2.wav.npy
Proccessed file ./audio/0_theo_2.wav
saved at ./spectrograms/0_theo_20.wav.npy
Proccessed file ./audio/0_theo_20.wav
saved at ./spectrograms/0_theo_21.wav.npy
Proccessed file ./audio/0_theo_21.wav
saved at ./spectrograms/0_theo_22.wav.npy
Proccessed file ./audio/0_theo_22.wav
saved at ./spectrograms/0_theo_23.wav.npy
Proccessed file ./audio/0_theo_23.wav
saved at ./spectrograms/0_theo_24.wav.npy
Proccessed file ./audio/0_theo_24.wav
saved at ./spectrograms/0_theo_25.wav.npy
Proccessed file ./audio/0_theo_25.wav
saved at ./spectrograms/0_theo_26.wav.npy
Proccessed file ./audio/0_theo_26.wav
saved at ./spectrograms/0_theo_27.wav.npy


saved at ./spectrograms/1_george_15.wav.npy
Proccessed file ./audio/1_george_15.wav
saved at ./spectrograms/1_george_16.wav.npy
Proccessed file ./audio/1_george_16.wav
saved at ./spectrograms/1_george_17.wav.npy
Proccessed file ./audio/1_george_17.wav
saved at ./spectrograms/1_george_18.wav.npy
Proccessed file ./audio/1_george_18.wav
saved at ./spectrograms/1_george_19.wav.npy
Proccessed file ./audio/1_george_19.wav
saved at ./spectrograms/1_george_2.wav.npy
Proccessed file ./audio/1_george_2.wav
saved at ./spectrograms/1_george_20.wav.npy
Proccessed file ./audio/1_george_20.wav
saved at ./spectrograms/1_george_21.wav.npy
Proccessed file ./audio/1_george_21.wav
saved at ./spectrograms/1_george_22.wav.npy
Proccessed file ./audio/1_george_22.wav
saved at ./spectrograms/1_george_23.wav.npy
Proccessed file ./audio/1_george_23.wav
saved at ./spectrograms/1_george_24.wav.npy
Proccessed file ./audio/1_george_24.wav
saved at ./spectrograms/1_george_25.wav.npy
Proccessed file ./audio/1_george_2

saved at ./spectrograms/1_lucas_19.wav.npy
Proccessed file ./audio/1_lucas_19.wav
saved at ./spectrograms/1_lucas_2.wav.npy
Proccessed file ./audio/1_lucas_2.wav
saved at ./spectrograms/1_lucas_20.wav.npy
Proccessed file ./audio/1_lucas_20.wav
saved at ./spectrograms/1_lucas_21.wav.npy
Proccessed file ./audio/1_lucas_21.wav
saved at ./spectrograms/1_lucas_22.wav.npy
Proccessed file ./audio/1_lucas_22.wav
saved at ./spectrograms/1_lucas_23.wav.npy
Proccessed file ./audio/1_lucas_23.wav
saved at ./spectrograms/1_lucas_24.wav.npy
Proccessed file ./audio/1_lucas_24.wav
saved at ./spectrograms/1_lucas_25.wav.npy
Proccessed file ./audio/1_lucas_25.wav
saved at ./spectrograms/1_lucas_26.wav.npy
Proccessed file ./audio/1_lucas_26.wav
saved at ./spectrograms/1_lucas_27.wav.npy
Proccessed file ./audio/1_lucas_27.wav
saved at ./spectrograms/1_lucas_28.wav.npy
Proccessed file ./audio/1_lucas_28.wav
saved at ./spectrograms/1_lucas_29.wav.npy
Proccessed file ./audio/1_lucas_29.wav
saved at ./spectro

...

saved at ./spectrograms/8_lucas_35.wav.npy
Proccessed file ./audio/8_lucas_35.wav
saved at ./spectrograms/8_lucas_36.wav.npy
Proccessed file ./audio/8_lucas_36.wav
saved at ./spectrograms/8_lucas_37.wav.npy
Proccessed file ./audio/8_lucas_37.wav
saved at ./spectrograms/8_lucas_38.wav.npy
Proccessed file ./audio/8_lucas_38.wav
saved at ./spectrograms/8_lucas_39.wav.npy
Proccessed file ./audio/8_lucas_39.wav
saved at ./spectrograms/8_lucas_4.wav.npy
Proccessed file ./audio/8_lucas_4.wav
saved at ./spectrograms/8_lucas_40.wav.npy
Proccessed file ./audio/8_lucas_40.wav
saved at ./spectrograms/8_lucas_41.wav.npy
Proccessed file ./audio/8_lucas_41.wav
saved at ./spectrograms/8_lucas_42.wav.npy
Proccessed file ./audio/8_lucas_42.wav
saved at ./spectrograms/8_lucas_43.wav.npy
Proccessed file ./audio/8_lucas_43.wav
saved at ./spectrograms/8_lucas_44.wav.npy
Proccessed file ./audio/8_lucas_44.wav
saved at ./spectrograms/8_lucas_45.wav.npy
Proccessed file ./audio/8_lucas_45.wav
saved at ./spectro

saved at ./spectrograms/8_theo_42.wav.npy
Proccessed file ./audio/8_theo_42.wav
saved at ./spectrograms/8_theo_43.wav.npy
Proccessed file ./audio/8_theo_43.wav
saved at ./spectrograms/8_theo_44.wav.npy
Proccessed file ./audio/8_theo_44.wav
saved at ./spectrograms/8_theo_45.wav.npy
Proccessed file ./audio/8_theo_45.wav
saved at ./spectrograms/8_theo_46.wav.npy
Proccessed file ./audio/8_theo_46.wav
saved at ./spectrograms/8_theo_47.wav.npy
Proccessed file ./audio/8_theo_47.wav
saved at ./spectrograms/8_theo_48.wav.npy
Proccessed file ./audio/8_theo_48.wav
saved at ./spectrograms/8_theo_49.wav.npy
Proccessed file ./audio/8_theo_49.wav
saved at ./spectrograms/8_theo_5.wav.npy
Proccessed file ./audio/8_theo_5.wav
saved at ./spectrograms/8_theo_6.wav.npy
Proccessed file ./audio/8_theo_6.wav
saved at ./spectrograms/8_theo_7.wav.npy
Proccessed file ./audio/8_theo_7.wav
saved at ./spectrograms/8_theo_8.wav.npy
Proccessed file ./audio/8_theo_8.wav
saved at ./spectrograms/8_theo_9.wav.npy
Procces

saved at ./spectrograms/9_george_6.wav.npy
Proccessed file ./audio/9_george_6.wav
saved at ./spectrograms/9_george_7.wav.npy
Proccessed file ./audio/9_george_7.wav
saved at ./spectrograms/9_george_8.wav.npy
Proccessed file ./audio/9_george_8.wav
saved at ./spectrograms/9_george_9.wav.npy
Proccessed file ./audio/9_george_9.wav
saved at ./spectrograms/9_jackson_0.wav.npy
Proccessed file ./audio/9_jackson_0.wav
saved at ./spectrograms/9_jackson_1.wav.npy
Proccessed file ./audio/9_jackson_1.wav
saved at ./spectrograms/9_jackson_10.wav.npy
Proccessed file ./audio/9_jackson_10.wav
saved at ./spectrograms/9_jackson_11.wav.npy
Proccessed file ./audio/9_jackson_11.wav
saved at ./spectrograms/9_jackson_12.wav.npy
Proccessed file ./audio/9_jackson_12.wav
saved at ./spectrograms/9_jackson_13.wav.npy
Proccessed file ./audio/9_jackson_13.wav
saved at ./spectrograms/9_jackson_14.wav.npy
Proccessed file ./audio/9_jackson_14.wav
saved at ./spectrograms/9_jackson_15.wav.npy
Proccessed file ./audio/9_jac

saved at ./spectrograms/9_nicolas_10.wav.npy
Proccessed file ./audio/9_nicolas_10.wav
saved at ./spectrograms/9_nicolas_11.wav.npy
Proccessed file ./audio/9_nicolas_11.wav
saved at ./spectrograms/9_nicolas_12.wav.npy
Proccessed file ./audio/9_nicolas_12.wav
saved at ./spectrograms/9_nicolas_13.wav.npy
Proccessed file ./audio/9_nicolas_13.wav
saved at ./spectrograms/9_nicolas_14.wav.npy
Proccessed file ./audio/9_nicolas_14.wav
saved at ./spectrograms/9_nicolas_15.wav.npy
Proccessed file ./audio/9_nicolas_15.wav
saved at ./spectrograms/9_nicolas_16.wav.npy
Proccessed file ./audio/9_nicolas_16.wav
saved at ./spectrograms/9_nicolas_17.wav.npy
Proccessed file ./audio/9_nicolas_17.wav
saved at ./spectrograms/9_nicolas_18.wav.npy
Proccessed file ./audio/9_nicolas_18.wav
saved at ./spectrograms/9_nicolas_19.wav.npy
Proccessed file ./audio/9_nicolas_19.wav
saved at ./spectrograms/9_nicolas_2.wav.npy
Proccessed file ./audio/9_nicolas_2.wav
saved at ./spectrograms/9_nicolas_20.wav.npy
Proccessed 

saved at ./spectrograms/9_yweweler_19.wav.npy
Proccessed file ./audio/9_yweweler_19.wav
saved at ./spectrograms/9_yweweler_2.wav.npy
Proccessed file ./audio/9_yweweler_2.wav
saved at ./spectrograms/9_yweweler_20.wav.npy
Proccessed file ./audio/9_yweweler_20.wav
saved at ./spectrograms/9_yweweler_21.wav.npy
Proccessed file ./audio/9_yweweler_21.wav
saved at ./spectrograms/9_yweweler_22.wav.npy
Proccessed file ./audio/9_yweweler_22.wav
saved at ./spectrograms/9_yweweler_23.wav.npy
Proccessed file ./audio/9_yweweler_23.wav
saved at ./spectrograms/9_yweweler_24.wav.npy
Proccessed file ./audio/9_yweweler_24.wav
saved at ./spectrograms/9_yweweler_25.wav.npy
Proccessed file ./audio/9_yweweler_25.wav
saved at ./spectrograms/9_yweweler_26.wav.npy
Proccessed file ./audio/9_yweweler_26.wav
saved at ./spectrograms/9_yweweler_27.wav.npy
Proccessed file ./audio/9_yweweler_27.wav
saved at ./spectrograms/9_yweweler_28.wav.npy
Proccessed file ./audio/9_yweweler_28.wav
saved at ./spectrograms/9_yweweler